<a href="https://colab.research.google.com/github/xechehot/sirius-2021-warehouse/blob/main/Maksim/beta_version1.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
def get_dist_for_1_path(sx, sy, block, data_for_ans): #data_for_ans - для ячейки храним какой заказ в ней
    x = sx
    y = sy
    dist = 0
    ans = []
    d = -1
    N = len(block)
    M = len(block[0])
    if (sy == 0): d = 1
    end = -1
    if (d == -1):
        for i in range(M):
            if (block[:, i].sum() == 0):
                end = i
            else: break
    else:
        for i in range(M - 1, -1, -1):
            if (block[:, i].sum() == 0):
                end = i
            else: break
    while True:
        if x < 0 or y < 0 or x >= N or y >= M or y == end: break
        if (block[:, y].sum() == 0):
            dist += 2
            y += d
            continue
        if x == N - 1: #идем вверх
            for nx in range(N - 1, -1, -1):
                if block[nx][y] != 0: 
                    ans.append(data_for_ans[nx][y])
        else: # идем вниз
            for nx in range(0, N):
                if (block[nx][y] != 0):
                    ans.append(data_for_ans[nx][y])
        dist += N
        dist += 2
        x = abs(x - (N - 1))
        y += d
    dist -= 2

    #if x == N - 1: dist += N
    return [get_normal_dist(ans), ans]




In [112]:
def get_next(places):
    for _ in range(3):
        i = rand(0, len(places) - 1)
        j = rand(0, len(places) - 1)
        if i == j:
            i = rand(0, len(places) - 1)
            j = rand(0, len(places) - 1)
        if i == j:
            i = rand(0, len(places) - 1)
            j = rand(0, len(places) - 1)
        if i > j: i, j = j, i
        if i != 0: places[i:j + 1] = places[j:i - 1:-1]
        else: places[i:j + 1] = places[j::-1]
        #places[i], places[j] = places[j], places[i]
    return places

def get_dist_with_annealing(places):
    t = 1
    dist = get_normal_dist(places)
    cnt = 0
    for _ in range(1000):
        new = get_next(places[::])
        new_dist = get_normal_dist(new)
        if (new_dist < dist):
            dist = new_dist
            places = new
        elif (new_dist > dist):
            #print(math.exp((dist - new_dist) / t))
            if (math.exp((dist - new_dist) / t) > rand(0, 10**6) / 10**6):
                places = new
                dist = new_dist
        t = t * 0.99
    return [dist, places]


In [113]:
from random import randint as rand
import numpy
import json
import math

In [88]:
! git clone https://github.com/xechehot/sirius-2021-warehouse.git

fatal: destination path 'sirius-2021-warehouse' already exists and is not an empty directory.


In [90]:
! cd sirius-2021-warehouse; git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/xechehot/sirius-2021-warehouse
   801ef67..ace90bb  main       -> origin/main
Updating 801ef67..ace90bb
Fast-forward
 data_sample/sample_3200_1.json | 20046 +++++++++++++++++++++++++++++++++++++++
 1 file changed, 20046 insertions(+)
 create mode 100644 data_sample/sample_3200_1.json


In [110]:
with open('sirius-2021-warehouse/data_sample/sample_3200_2.json') as f:
    sample_data = json.load(f)

In [114]:
rows = sample_data['warehouse']['meta']['rows']
sections = sample_data['warehouse']['meta']['sections']
block_x = sample_data['warehouse']['meta']['block_x']
block_y = sample_data['warehouse']['meta']['block_y']
floors = sample_data['warehouse']['meta']['floors']
batch = sample_data['batch_size']
b_sz = 32
N = rows // 2 + 1
M = sections + 1

In [134]:
def get_places(cnt_products, product_place): #ищу ячейки, куда нужно зайти
    ans = []
    for id in product_set:
        product_place[id].sort(key = lambda a: -a['count']) #????????
    for id in product_set:
        have = cnt_products[id]
        ind = 0
        while (have > 0 and ind < len(product_place[id])):
            sample_data['warehouse']['stock'].remove(product_place[id][ind])
            if (product_place[id][ind]['count'] >= have):
                ans.append([product_place[id][ind]['p'], have])
                product_place[id][ind]['count'] -= have
                sample_data['warehouse']['stock'].append(product_place[id][ind])
                have = 0
            else:
                ans.append([product_place[id][ind]['p'], product_place[id][ind]['count']])
                have -= product_place[id][ind]['count']
            ind += 1
    return ans

In [116]:
def distribute_data(places): #разделяю ячейки по этажам, блокам 
    ans = [[[[] for i1 in range(block_y)] for i2 in range(block_x)] for i3 in range(floors)]
    for place in places:
        ans[place[0]['floor'] - 1][place[0]['block_x'] - 1][place[0]['block_y'] - 1].append(place)
    return ans

In [117]:
def conv_place(place):
    return [place['row'] // 2, place['section']]

In [118]:
def dist_to_start(place): 
    ans = 0
    dy = int(abs(block_y // 2 + 0.5 - place['block_y']) + 0.5) - 1
    ans += dy * (rows + 2) #расстояние по горизонтали к центру
    ans += (block_x - place['block_x']) * (sections + 2) #расстояние по вертикали к низу
    if (place['block_y'] <= block_y // 2): ans += (rows - place['row']) #расстояние внутри блока
    else: ans += place['row']
    ans += (sections + 1 - place['section'])
    return ans

In [119]:
def dist_to_tape(place):
    ans = place['section'] + (place['block_x'] - 1) * (sections)
    return ans

In [120]:
def get_ans_for_1_block(data, sx, sy):
    block = numpy.zeros((N, M), int)
    list_of_products = [[-1 for i in range(M)] for j in range(N)]
    #print(data)
    bx, by = -1, -1
    for place, cnt in data:
        x, y = conv_place(place)
        bx = place['block_x']
        by = place['block_y']
        block[x, y] += cnt
        list_of_products[x][y] = [place, cnt]
    s_dist, s_paths = get_dist_for_1_path(sx, sy, block, list_of_products)
    dist, paths = get_dist_with_annealing(s_paths)
    ans = []
    cur = []
    sum = 0
    #print(paths)
    for place, cnt in paths:
        if (sum + cnt <= 32):
            cur.append([place, cnt])
            if sum == 0:
                dist += dist_to_start(place)
            sum += cnt
        else:
            exist = cnt
            while exist > 0:
                now = min(exist, 32 - sum)
                cur.append([place, now])
                if sum == 0:
                    dist += dist_to_start(place)
                sum += now
                exist -= now
                if (sum == 32):
                    dist += dist_to_tape(place) * 2
                    dist += dist_to_start(place)
                    ans.append(cur)
                    cur = []
                    sum = 0
        if (sum == 32):
            dist += dist_to_tape(place) * 2
            dist += dist_to_start(place)
            ans.append(cur)
            cur = []
            sum = 0
    if (cur != []):
        dist += dist_to_tape(cur[-1][0]) * 2
        dist += dist_to_start(cur[-1][0])
        ans.append(cur)
        cur = []
        sum = 0
    return [ans, dist]
    



In [121]:
def get_dist_2_places(p1, p2):
    ans = 0
    if (p1['block_x'] > p2['block_x']): p1, p2 = p2, p1
    ans += (p2['block_x'] - p1['block_x']) * (sections + 2)
    ans += min(p1['section'] + p2['section'] + 2, 2 * sections - p1['section'] - p2['section'] + 2)
    if (p1['block_y'] > p2['block_y']): p1, p2 = p2, p1
    ans += (rows - p1['row']) + p2['row']
    ans += (p2['block_y'] - p1['block_y'] - 1) * (rows + 2) + 2
    return ans

In [122]:
def get_path_len(path):
    ans = 0
    for place, cnt in path:
        ans += cnt
    return ans

def get_ans_for_1_floor(paths): # возвращает пару ответ и изменение dist
    delta = 0
    res = []
    ans = [[] for i in range(block_x * block_y)]
    for path in paths:
        if (get_path_len(path) == 32):
            res.append(path)
            continue
        num = (path[0][0]['block_x'] - 1) * block_y
        if path[0][0]['block_x'] == 1: num += path[0][0]['block_y'] - 1
        else: num += (block_y - path[0][0]['block_y'])
        ans[num] = path
    last = []
    for path in ans:
        if path == []: continue
        if last == []:
            last = path
        else:
            if (get_path_len(last) + get_path_len(path) <= 32):
                add = get_dist_2_places(last[-1][0], path[0][0])
                sub = dist_to_start(last[-1][0]) + dist_to_start(path[0][0]) + dist_to_tape(last[-1][0]) * 2
                if (add < sub):
                    delta += add
                    delta -= sub
                    for elem in path: last.append(elem)
                else:
                    res.append(last)
                    last = path
            else:
                res.append(last)
                last = path
    if (last != []): res.append(last)
    return [res, delta]






In [132]:
product_place = {}
cnt_products = {}
product_set = set()
block_popular = [[0 for i in range(block_x * block_y)] for j in range(floors)]

In [135]:
def prepair(batch_num):
    ans = 0
    for product in sample_data['warehouse']['stock']:
        if product_place.get(product['id']) == None:
            product_place[product['id']] = []
        product_place[product['id']].append(product)
    for i in range(batch_num * batch, min((batch_num + 1) * batch, len(sample_data['orders']))):
        order = sample_data['orders'][i]
        for product in order['items']:
            product_set.add(product['id'])
            if (cnt_products.get(product['id']) == None):
                cnt_products[product['id']] = 0
            ans += product['count']
            cnt_products[product['id']] += product['count']
    return ans

In [125]:
def get_normal_dist(path):
    ans = 0
    ans += dist_to_start(path[0][0]) + dist_to_start(path[-1][0])
    ans += dist_to_tape(path[-1][0]) * 2
    for i in range(len(path) - 1):
        ans += get_dist_2_places(path[i][0], path[i + 1][0])
    return ans

In [126]:
def get_where_exist(order):
    cnt1 = 0
    cnt2 = 0
    for product in order:
        place = product_place[product['id']][0]
        if (place['p']['block_y'] <= block_y // 2):
            cnt1 += product['count']
        cnt2 += product['count']
    return cnt1 / cnt2

def sort_orders():
    sample_data['orders'].sort(key = lambda x: get_where_exist(x['items']))


In [138]:
def get_place_block(place):
    return (place['block_x'] - 1) * block_y + place['block_y'] - 1

def count_blocks_popular():
    for order in sample_data['orders']:
        cnt2 = 0
        for product in order['items']:
            cnt2 += product['count']
        for product in order['items']:
            for place in product_place[product['id']]:
                block_popular[place['floor'] - 1] += min(place['count'], product['count']) / cnt2


In [146]:
sum_dist = 0
cnt = 0
for order in sample_data['orders']:
    for product in order['items']:
        cnt += product['count']
prepair(0)
print(product_place)
raise Exception()
sort_orders()
for i in range((len(sample_data['orders']) - 1) // batch + 1):
    cnt_products = {}
    product_set = set()
    product_place = {}
    block_popular = [[0 for i in range(block_x * block_y)] for j in range(floors)]
    prepair(i)
    print(product_place)
    #count_blocks_popular()
    break
    places = get_places(cnt_products, product_place)
    dis = distribute_data(places)
    dist = 0
    ans_for_floors = [[] for i in range(floors)]
    for floor in range(floors):
        for bx in range(block_x):
            for by in range(block_y):
                if (dis[floor][bx][by] == []): continue
                sx = N - 1
                if (by >= 2): sx = 0
                p_ans, p_dist = get_ans_for_1_block(dis[floor][bx][by], sx, 0)
                dist += p_dist
                for elem in p_ans: 
                    ans_for_floors[floor].append(elem)
    ans = []
    for i in range(floors):
        p_ans, delta = get_ans_for_1_floor(ans_for_floors[i])
        for elem in p_ans: ans.append(elem)
        dist += delta
    #print(dist, delta)
    dist_ans = 0
    for path in ans:
        dist_ans += get_normal_dist(path)
    #print(dist_ans)
    sum_dist += dist_ans
print(sum_dist / cnt)


{21261: [{'count': 1, 'id': 21261, 'p': {'block_x': 1, 'block_y': 1, 'floor': 2, 'row': 11, 'section': 18}}, {'count': 1, 'id': 21261, 'p': {'block_x': 1, 'block_y': 1, 'floor': 2, 'row': 11, 'section': 18}}, {'count': 1, 'id': 21261, 'p': {'block_x': 1, 'block_y': 1, 'floor': 2, 'row': 11, 'section': 18}}, {'count': 1, 'id': 21261, 'p': {'block_x': 1, 'block_y': 1, 'floor': 2, 'row': 11, 'section': 18}}, {'count': 1, 'id': 21261, 'p': {'block_x': 1, 'block_y': 1, 'floor': 2, 'row': 11, 'section': 18}}, {'count': 1, 'id': 21261, 'p': {'block_x': 1, 'block_y': 1, 'floor': 2, 'row': 11, 'section': 18}}], 156240: [{'count': 3, 'id': 156240, 'p': {'block_x': 2, 'block_y': 1, 'floor': 2, 'row': 16, 'section': 14}}, {'count': 2, 'id': 156240, 'p': {'block_x': 2, 'block_y': 1, 'floor': 2, 'row': 3, 'section': 1}}, {'count': 2, 'id': 156240, 'p': {'block_x': 1, 'block_y': 2, 'floor': 1, 'row': 19, 'section': 16}}, {'count': 3, 'id': 156240, 'p': {'block_x': 2, 'block_y': 1, 'floor': 2, 'row': 

Exception: ignored